In [1]:
import pandas as pd
import pvlib

# import numpy as np
# import pickle 
# from datetime import timedelta
# import time
# from dateutil.relativedelta import relativedelta
# from dateutil.parser import parse

# from herbie import Herbie
# from herbie import FastHerbie

# from fractions import Fraction

# # import cartopy.crs as ccrs
# import matplotlib.pyplot as plt

# # from paint.standard2 import cm_tmp, cm_wind, cm_wave_height, cm_pcp
# # from toolbox import EasyMap, pc, ccrs
# import xarray as xr



# from herbie_gefs_tools import get_many_tcc_fcasts_fastherbie, get_tcc_fcast, get_tcc_fcasts_fastherbie
# from herbie_hrrr_tools import hrrr_ds_to_power, get_hrrr_fcasts_fastherbie, get_many_hrrr_fcasts_fastherbie

# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.metrics import mean_pinball_loss, mean_squared_error

# import gc

In [2]:
plants_df = pd.read_csv('pv_fleet_metadata_rev3.csv') # read in as a Pandas DataFrame
plants_df = plants_df[plants_df['ac_capacity']>=10].copy().reset_index()
num_plants = len(plants_df.index) # number of plants

In [3]:
p_ac_all_act_2021 = pd.read_pickle('data_files/p_ac_all_act_2021.pkl')
p_ac_all_cs_2021 = pd.read_pickle('data_files/p_ac_all_cs_2021.pkl')
p_ac_all_fcast_2021 = pd.read_pickle('data_files/p_ac_all_fcast_2021.pkl')
p_ac_all_fcast_max_2021 = pd.read_pickle('data_files/p_ac_all_fcast_max_2021.pkl')
p_ac_all_fcast_min_2021 = pd.read_pickle('data_files/p_ac_all_fcast_min_2021.pkl')

p_ac_all_act_2022 = pd.read_pickle('data_files/p_ac_all_act_2022.pkl')
p_ac_all_cs_2022 = pd.read_pickle('data_files/p_ac_all_cs_2022.pkl')
p_ac_all_fcast_2022 = pd.read_pickle('data_files/p_ac_all_fcast_2022.pkl')
p_ac_all_fcast_max_2022 = pd.read_pickle('data_files/p_ac_all_fcast_max_2022.pkl')
p_ac_all_fcast_min_2022 = pd.read_pickle('data_files/p_ac_all_fcast_min_2022.pkl')

p_ac_all_act_2023 = pd.read_pickle('data_files/p_ac_all_act_2023.pkl')
p_ac_all_cs_2023 = pd.read_pickle('data_files/p_ac_all_cs_2023.pkl')
p_ac_all_fcast_2023 = pd.read_pickle('data_files/p_ac_all_fcast_2023.pkl')
p_ac_all_fcast_max_2023 = pd.read_pickle('data_files/p_ac_all_fcast_max_2023.pkl')
p_ac_all_fcast_min_2023 = pd.read_pickle('data_files/p_ac_all_fcast_min_2023.pkl')

In [4]:
w_avg_tcc = pd.read_pickle('data_files/w_avg_tcc_2021-2023.pkl')

Concatenate data across years:

In [5]:
p_ac_all_act = pd.concat([p_ac_all_act_2021, p_ac_all_act_2022, p_ac_all_act_2023], axis=0)
p_ac_all_cs = pd.concat([p_ac_all_cs_2021, p_ac_all_cs_2022, p_ac_all_cs_2023], axis=0)
p_ac_all_fcast = pd.concat([p_ac_all_fcast_2021, p_ac_all_fcast_2022 ,p_ac_all_fcast_2023], axis=0)
p_ac_all_fcast_max = pd.concat([p_ac_all_fcast_max_2021, p_ac_all_fcast_max_2022, p_ac_all_fcast_max_2023], axis=0)
p_ac_all_fcast_min = pd.concat([p_ac_all_fcast_min_2021, p_ac_all_fcast_min_2022, p_ac_all_fcast_min_2023], axis=0)

# p_ac_all_act = pd.concat([p_ac_all_act_2021, p_ac_all_act_2022], axis=0)
# p_ac_all_cs = pd.concat([p_ac_all_cs_2021, p_ac_all_cs_2022], axis=0)
# p_ac_all_fcast = pd.concat([p_ac_all_fcast_2021, p_ac_all_fcast_2022], axis=0)
# p_ac_all_fcast_max = pd.concat([p_ac_all_fcast_max_2021, p_ac_all_fcast_max_2022], axis=0)
# p_ac_all_fcast_min = pd.concat([p_ac_all_fcast_min_2021, p_ac_all_fcast_min_2022], axis=0)

Rename columns:

In [6]:
p_ac_all_act.rename(columns={'power_ac': 'p_ac_all_act'}, inplace=True)
p_ac_all_cs.rename(columns={'power_ac': 'p_ac_all_cs'}, inplace=True)
p_ac_all_fcast.rename(columns={'power_ac': 'p_ac_all_fcast'}, inplace=True)
p_ac_all_fcast_max.rename(columns={'power_ac': 'p_ac_all_fcast_max'}, inplace=True)
p_ac_all_fcast_min.rename(columns={'power_ac': 'p_ac_all_fcast_min'}, inplace=True)

Combine into a single DataFrame:

In [7]:
# list of dataframes to combiner first
dfs = [
    p_ac_all_fcast,
    p_ac_all_fcast_max,
    p_ac_all_fcast_min,
    p_ac_all_cs,
    p_ac_all_act,
    w_avg_tcc.tcc_std,
    ]

# combine all dataframes and drop na rows
df = pd.concat(dfs, axis=1).dropna()

df.index.name = 'time_center_labeled'

Calculate normalized Max-Min:

In [8]:
df['max_min_norm'] = (df['p_ac_all_fcast_max'] - df['p_ac_all_fcast_min'])/df['p_ac_all_cs']

Calculate error statistics:

In [9]:
approx_nameplate = df.p_ac_all_cs.max() # use max of clear sky power as proxy for nameplate rating
df['abs_err'] = (df.p_ac_all_fcast - df.p_ac_all_act).abs()
df['norm_abs_err'] = df.abs_err/approx_nameplate
df['csi_norm_abs_err'] = df.abs_err/df.p_ac_all_cs

Normalize to clear sky by calculating clear sky index (CSI) of actual and forecasted power:

In [10]:
df['csi_actual'] = df['p_ac_all_act']/df['p_ac_all_cs']
df['csi_fcast'] = df['p_ac_all_fcast']/df['p_ac_all_cs']
# df['csi_fcast_max'] = df['p_ac_all_fcast_max']/df['p_ac_all_cs']
# df['csi_fcast_min'] = df['p_ac_all_fcast_min']/df['p_ac_all_cs']

In [11]:
# add month, hour, solar azimuth, and solar elevation
latitude, longitude = 32.8 , -83.6
solpos = pvlib.solarposition.get_solarposition(time=df.index,latitude=latitude, longitude=longitude)

df['hour'] = df.index.hour
df['month'] = df.index.month
df['el'] = solpos.apparent_elevation
df['zen'] = solpos.apparent_zenith
df['az'] = solpos.azimuth

In [12]:
df.head()

,p_ac_all_fcast,p_ac_all_fcast_max,p_ac_all_fcast_min,p_ac_all_cs,level_0,p_ac_all_act,tcc_std,max_min_norm,abs_err,norm_abs_err,csi_norm_abs_err,csi_actual,csi_fcast,hour,month,el,zen,az
time_center_labeled,,,,,,,,,,,,,,,,,,
2021-01-01 09:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.657303,NaN,0.000000,0.00000,NaN,NaN,NaN,9,1,-44.725579,134.725579,90.865286
2021-01-01 10:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.346134,NaN,0.000000,0.00000,NaN,NaN,NaN,10,1,-32.169594,122.169594,98.221778
2021-01-01 11:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.346134,NaN,0.000000,0.00000,NaN,NaN,NaN,11,1,-19.832171,109.832171,105.126225
2021-01-01 12:00:00+00:00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.346134,NaN,0.000000,0.00000,NaN,NaN,NaN,12,1,-7.896291,97.896291,112.278339
2021-01-01 13:00:00+00:00,46.237118,146.468144,4.036621,827.123924,0.0,32.438303,4.919961,0.172201,13.798815,0.00372,0.016683,0.039218,0.055901,13,1,3.619760,86.380240,120.212465


In [13]:
df.to_pickle('data_files/full_2021_2023_dataset.pkl')